# Intro and Overview

# Data and Sources

# Serbian Municipalities Overview

To be included:

Basic:
- total number, breakdown by city, city districts, rest + kosovo (our focus is on rest)
- distributions by population
- distribution by land size
- distribution by pop density

Wealth:
- average wages and/or budget per capita
- employment

